# Principal Components Echo State Networks

## Import

In [21]:
from numpy import *
from matplotlib import pyplot as plt
from scipy.sparse import rand as sprand
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigs as speigs

## Helper functions

In [22]:
class PCESN():
    def __init__(self,nInputUnits,nReservoirUnits,nOutputUnits,spectralRadius,sigma2=0.1, phi2 = 1,eta):
        print('Creating PC-ESN...')
        ### STRUCTURE
        self.nInputUnits = nInputUnits
        self.nReservoirUnits = nReservoirUnits
        self.nOutputUnits = nOutputUnits
        
        self.sigma2 = sigma2
        self.phi2 = phi2
        self.eta = eta # learning rate
        
        # initalize input sum and mean
        self.inputMean = 0 
        self.inputAbs = 0.00001
        self.i = 0

        ### INITIALIZE WEIGHTS JAEGER (Sparse reservoir weights) (Polydoros et. al. Algorithm 1)
        success = 0                                             
        while success == 0: # following block might fail
            try:
                self.Wres = sprand(nReservoirUnits, nReservoirUnits, density=10/nReservoirUnits)
                self.Wres = self.Wres.toarray()
                self.Wres[self.Wres!=0] -= 0.5 # modify only nonzero elements
                self.Wres = csr_matrix(self.Wres) # back to sparse
                maxVal = max(abs(speigs(A=self.Wres, k=1, which='LM')[0]))
                self.Wres /= maxVal
                success = 1
            except:
                success = 0   
        self.Wres *= self.spectralRadius
        
        self.Win = eye(nInputUnits)
        self.Wself = (2.0*random.rand(nReservoirUnits, nInputUnits)-1.0) # init not mentioned???
        self.Wfb = (2.0 * random.rand(nReservoirUnits, nOutputUnits)- 1.0)
        self.Wout = (2.0 * random.rand(nOutputUnits,nReservoirUnits)- 1.0) # init not mentioned???
        self.Wdir = (2.0 * random.rand(nOutputUnits,nInputUnits)- 1.0)
        
        self.Wtrain = zeros((nOutputUnits, nInputUnits + nReservoirUnits))

        self.V = self.sigma2 * eye(nInputUnits + nReservoirUnits)
        
        print('Successful!')
        
        def train(self, inputSample,targetSample):
            # Normalize and center input
            self.inputMean = self.inputMean + (inputSample-self.inputMean)/(self.i+1)
            inputSample -= self.inputMean
            self.inputAbs = maximum(self.inputAbs, absolute(inputSample))
            inputSample /= self.inputAbs
            self.i += 1 # update index

            self.s = tanh(self.Win @ inputSample) # update self-organized layer
            self.r = tanh(self.Wres @ self.r + self.Wself @ self.s + self.Wfb @ self.o)
            self.o = self.Wtrain @ self.ct
            
            Vprev = self.V
            self.V = linalg.inv(linalg.inv(Vprev) + (1/self.sigma_2) * c @ c.T)
            
            a = self.V @ np.linalg.inv(V_prev) @ self.W_train.T
            b = 1/self.sigma_2 * self.V @ c_t @ tau.T
            
            self.Wtrain = np.sum([a.T, b.T], axis=0)
        
            # Calculate GHL update 
            triang = tril(self.s @ self.s.T)
            dWin = self.eta*(self.s@inputSample.T - triang @ self.Win)

            self.Win += dWin # update Win matrix

## Useless(?) stuff

In [ ]:
# Init
self.noiseLevel = 0.0
self.leakingRate = 1
self.forgetPoints = 100
self.reg = 1e-5 # If ridge regression!!! (something else than 1)

self.RLS_lambda = 0.9999995
self.RLS_delta = 0.000001

self.trained = 0
self.pseudo = True

# init default
self.inputScaling = ones((nInputUnits, 1)) # MAKE SURE INPUT IS NORMALIZED!!!
self.inputShift = zeros((nInputUnits, 1))
self.teacherScaling = ones((nOutputUnits, 1)) # DOES TEACHER SCALING MAKE ANY DIFFERENCE???
self.teacherShift = zeros((nOutputUnits, 1))
self.teacherForcing = True # Desired output y_teacher instead of predicted y -> Ridge regression!!!  
self.feedbackScaling = zeros((nOutputUnits, 1))